## Setup Visdom

Install it with:

`pip install visdom`

Start the server:

`python -m visdom.server`

Visdom now can be accessed at http://localhost:8097 in the browser.

To monitor the training progress of LDA live in Visdom browser, you can set `viz=True` in the LDA function call.

`model = ldamodel.LdaModel(corpus=corpus, id2word=dictionary, passes=50, viz=True)`

When the model is set for training, you can open http://localhost:8097 to see the training progress.


## LDA Training Visualization

There are four types of graphs which are plotted for LDA:

**Coherence**

Coherence is a measure used to evaluate topic models. A good model will generate coherent topics, i.e., topics         with high topic coherence scores. Good topics are topics that can be described by a short label based on the topic     terms they spit out. 

<img src="Coherence.gif">

Now, this graph along with the others explained below, can be used to decide if it's time to stop the training. We     can see if the value stops changing after some epochs and that we are able to get the highest possible coherence       of our model.  


**Perplexity**

Perplexity is a measurement of how well a probability distribution or probability model predicts a sample. In LDA, topics are described by a probability distribution over vocabulary words. So, perplexity can be used to compare probabilistic models like LDA.

<img src="Perplexity.gif">

For a good model, perplexity should be as low as possible.


**Topic Difference**

Topic Diff calculates the distance between two LDA models. This distance is calculated based on the topics, by either using their probability distribution over vocabulary words (kullback_leibler, hellinger) or by simply using the common vocabulary words between the topics from both model.

<img src="Diff.gif">

In the heatmap, X-axis define the Epoch no. and Y-axis define the distance between the identical topic from consecutive epochs. For ex. a particular cell in the heatmap with values (x=3, y=5, z=0.4) represent the distance(=0.4) between the topic 5 from 3rd epoch and topic 5 from 2nd epoch. With increasing epochs, the distance between the identical topics should decrease.
  
  
**Convergence**

Convergence is the sum of the difference between all the identical topics from two consecutive epochs. It is basically the sum of column values in the heatmap above.

<img src="Convergence.gif">

The model is said to be converged when the convergence value stops descending with increasing epochs.